## Layout Planning

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('120s PDM Data.xlsx',sheet_name = None)

### 讀進flow

In [20]:
xls = pd.ExcelFile('120s PDM Data.xlsx')
xls.sheet_names 
flow = data.get('Flow')
flow.head()

,TRAVELER,TRAV DESC,STEP NAME,RECIPE NAME,WS,AREA,RPT MINS,WS LOAD PCT,DISPATCH RPT,SUBSET,...,SUB-FAMILY,PROCESS,DENSITY,RPT VALIDATION STATUS,MODELED UTIL,AVG LOADSIZE,MODELED AVAIL,LOST TIME,RPT BASIS,MODELED CHAMBERS
0,V00H,100S PC,0000-00 START LOT START,PC LOT START,11-3START_WAFERS,F11 GENERAL,0.43,100.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,1.00,25.0,1.00,0.0,NaN,1.0
1,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU31_DHF_4,F11 WET PROCESS,0.70,0.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
2,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU3_DHF100,F11 WET PROCESS,0.70,5.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
3,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU31_DHF100,F11 WET PROCESS,0.35,95.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
4,V00H,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,CMP_100S_BUFF,11-3EBRA_FRXS_BUFF,F11 CMP,1.12,100.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.75,1.0,0.86,0.0,1.0,1.0


In [21]:
flow = flow[flow['TRAV DESC'] == '120S PC']
flow.shape[0]

6326

In [22]:
#flow_dup = flow.drop_duplicates(subset = ['WS'],keep = 'first',inplace=False)
#flow_dup.head(400)
#flow_dup = flow_dup.(flow_dup[(flow_dup['AREA'].isin(['F11 GENERAL']))])
#flow_dup.head()

### 索引全部經過Workstation名稱

In [23]:
flow = flow[flow.AREA != 'F11 GENERAL']
flow.head()
step = flow['WS'][:-1].tolist()
#print(step)


### 獲取WS與Equipment ID對應關係

In [24]:
Qty = data.get('Qty')

In [25]:
WS_EQ = pd.concat([Qty['WS NAME'],Qty['EQUIP ID']],axis=1)
WS_EQ.head() #機台對應 WS
WS_NAME = WS_EQ['WS NAME'].tolist()
#print(WS_NAME)

In [26]:
WS_EQ.head()

,WS NAME,EQUIP ID
0,11-3ABON_ZFB_N2P,AN2PB80101
1,11-3ABON_ZFB_N2P,AN2PB80102
2,11-3ABON_ZFB_N2P,AN2PB80103
3,11-3ABON_ZFB_N2P,AN2PB80104
4,11-3ABON_ZFB_N2P,AN2PB80105


### 投片數

In [27]:
wafer = input('Please enter the quantity of wafers:')
wafer_m = []
for i in step:
    if i in WS_NAME:
        w = round(int(wafer)/WS_EQ.groupby(['WS NAME']).size()[i],2)
        wafer_m.append(w)
    else:
        wafer_m.append(0)
    #print(i)
#print ('Number of wafers in each machine:',wafer_m)

'''for i in step:
    if i not in WS_NAME:
        step.remove(i)
    else:
        None
print(step)'''

Please enter the quantity of wafers:600


'for i in step:\n    if i not in WS_NAME:\n        step.remove(i)\n    else:\n        None\nprint(step)'

### 找出WS有多少台機台，並平均分配wafer數量

In [30]:
col = ['3','4','7','8','9','C','D','E']
#print(wafer_in_each_machine)
wafer_in_each_machine = pd.DataFrame(columns = col)
machine_num = pd.DataFrame(columns = col)
for i in step:
    f1 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '3')].shape[0]
    f2 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '4')].shape[0]
    f3 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '7')].shape[0]
    f4 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '8')].shape[0]
    f5 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '9')].shape[0]
    f6 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'C')].shape[0]
    f7 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'D')].shape[0]
    f8 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'E')].shape[0]
    machine = np.array([f1,f2,f3,f4,f5,f6,f7,f8])
    #print(machine)
    machine_D = pd.DataFrame([machine],columns = col)
    machine_num = machine_num.append(machine_D)
    for j in range(8):
        if i in WS_NAME:
            num = wafer_m[step.index(i)]*machine[j]
        else:
            num = 0
        machine[j] = num
    machine = pd.DataFrame([machine],columns = col)
    #print(machine)
    wafer_in_each_machine = wafer_in_each_machine.append(machine)

In [31]:
#machine_num.head()

In [32]:
wafer_in_each_machine['WS NAME'] = step 
wafer_in_each_machine.set_index('WS NAME',inplace = True)
wafer_in_each_machine.head(20)

,3,4,7,8,9,C,D,E
WS NAME,,,,,,,,
11-3EBRA_FRXS_BUFF1,0,0,0,0,0,600,0,0
11-3TEL_303I_PADOX,0,0,150,0,0,450,0,0
11-3KLAT_FX200_THK,300,0,0,0,0,300,0,0
11-3KLAT_F5X_THK,240,0,240,0,0,120,0,0
11-3KLAT_FX100_THK,0,0,600,0,0,0,0,0
11-3KLAT_F5X_THK_1,100,0,200,0,0,300,0,0
11-3KLAT_FX200_THK_1,0,0,342,0,0,257,0,0
11-3DNS_SS32_SCRB_DS,240,0,240,0,0,120,0,0
11-3DNS_SS33_SCRB_DS,0,0,0,0,0,600,0,0


In [33]:
wafer_in_each_machine.shape
w = wafer_in_each_machine.values
#print(w)
w.shape

(6108, 8)

In [34]:
import itertools
col1 = ['3','4','7','8','9','C','D','E']
print(list(itertools.product(col, col1)))

[('3', '3'), ('3', '4'), ('3', '7'), ('3', '8'), ('3', '9'), ('3', 'C'), ('3', 'D'), ('3', 'E'), ('4', '3'), ('4', '4'), ('4', '7'), ('4', '8'), ('4', '9'), ('4', 'C'), ('4', 'D'), ('4', 'E'), ('7', '3'), ('7', '4'), ('7', '7'), ('7', '8'), ('7', '9'), ('7', 'C'), ('7', 'D'), ('7', 'E'), ('8', '3'), ('8', '4'), ('8', '7'), ('8', '8'), ('8', '9'), ('8', 'C'), ('8', 'D'), ('8', 'E'), ('9', '3'), ('9', '4'), ('9', '7'), ('9', '8'), ('9', '9'), ('9', 'C'), ('9', 'D'), ('9', 'E'), ('C', '3'), ('C', '4'), ('C', '7'), ('C', '8'), ('C', '9'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('D', '3'), ('D', '4'), ('D', '7'), ('D', '8'), ('D', '9'), ('D', 'C'), ('D', 'D'), ('D', 'E'), ('E', '3'), ('E', '4'), ('E', '7'), ('E', '8'), ('E', '9'), ('E', 'C'), ('E', 'D'), ('E', 'E')]


### 照比例分配搬運量，assign weight

In [35]:
a = range(8)
m = []
for k in range(len(step)-1):
    for j in a:
        for i in a:
            n = max(w[k,j]-w[k+1,j],0)*w[k+1,a]//(int(wafer)-w[k+1,j]+1) #避免分母為零
        m.append(n)      
step_m = np.array(m)
#print(step_m)
#print(step_m.shape)
carry = np.vsplit(step_m, len(step)-1)
for i in range(len(step)-1):
    for j in range(8):
        carry[i][j][j] = min(w[i,j],w[i+1,j])
        carry = np.array(carry)
#print(carry)


In [36]:
def weight(distance,carry,step):
    carry_amount = pd.DataFrame(columns = list(itertools.product(col, col1)))
    total_carry = []
    step_carriage = []
    for i in range(len(step)-1):
        carry_weight = np.multiply(carry[i],distance)
        total_carry_step = np.sum(carry_weight)-np.trace(carry_weight)
        total_carry.append(total_carry_step)
        carry_weight = carry_weight.reshape(1,len(col)**2)
        carry_weight = pd.DataFrame(carry_weight,columns = list(itertools.product(col, col1)))
        carry_amount = carry_amount.append(carry_weight)
    carry_amount.insert(0,'From',step[:len(step)-1])
    carry_amount.insert(1,'To',step[1:len(step)])
    carry_amount['Step carriage'] = total_carry
    carry_amount.index = range(len(carry_amount))
    #print ('Total carriage:',sum(total_carry))
    return carry_amount
    
weight(distance = np.ones((8,8)),carry = carry,step = step).head()

,From,To,"(3, 3)","(3, 4)","(3, 7)","(3, 8)","(3, 9)","(3, C)","(3, D)","(3, E)",...,"(D, E)","(E, 3)","(E, 4)","(E, 7)","(E, 8)","(E, 9)","(E, C)","(E, D)","(E, E)",Step carriage
0,11-3EBRA_FRXS_BUFF1,11-3TEL_303I_PADOX,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,149.0
1,11-3TEL_303I_PADOX,11-3KLAT_FX200_THK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,297.0
2,11-3KLAT_FX200_THK,11-3KLAT_F5X_THK,240,0,39,0,0,19,0,0,...,0,0,0,0,0,0,0,0,0,236.0
3,11-3KLAT_F5X_THK,11-3KLAT_FX100_THK,0,0,239,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,358.0
4,11-3KLAT_FX100_THK,11-3KLAT_F5X_THK_1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,398.0
5,11-3KLAT_F5X_THK_1,11-3KLAT_FX200_THK_1,0,0,56,0,0,42,0,0,...,0,0,0,0,0,0,0,0,0,140.0
6,11-3KLAT_FX200_THK_1,11-3DNS_SS32_SCRB_DS,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,236.0
7,11-3DNS_SS32_SCRB_DS,11-3DNS_SS33_SCRB_DS,0,0,0,0,0,239,0,0,...,0,0,0,0,0,0,0,0,0,478.0
8,11-3DNS_SS33_SCRB_DS,11-3TEL_NS3S_SCRB_DS,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,373.0
9,11-3TEL_NS3S_SCRB_DS,11-3TEL_303I_TEOS,0,0,6,0,0,68,0,0,...,0,0,0,0,0,0,0,0,0,319.0


In [23]:
#weight(distance = np.ones((8,8)),carry = carry,step = step).to_excel('flow.xlsx') 